In [ ]:
# encoding: utf-8
# 导入fluent_corba下的CORBA接口类和其他必要模块
from fluent_corba import CORBA
import time
import pathlib
import os, sys
import subprocess
import numpy as np 
import linecache
import csv


# 定义Fluent的启动位置，例如2020R1版本
ansysPath = pathlib.Path(os.environ["AWP_ROOT201"])
fluentExe = str(ansysPath/"fluent"/"ntbin"/"win64"/"fluent.exe")
# 定义工作目录
workPath = pathlib.Path(r"F:\Fluent_Python")
aasFilePath = workPath/"aaS_FluentId.txt"
# 服务器会话连接之前，清除工作目录下存在的aaS*.txt文件
for file in workPath.glob("aaS*.txt"): file.unlink()
# 启动线程调用Fluent软件
fluentProcess = subprocess.Popen(f'"{fluentExe}" 2ddp -aas',
                                 shell=True, cwd=str(workPath))
# 监控aaS_FluentId.txt文件生成，等待corba连接
while True:
    try:
        if not aasFilePath.exists():
            time.sleep(0.2)
            continue
        else:
            if "IOR:" in aasFilePath.open("r").read():
                break
    except KeyboardInterrupt: sys.exit()
# 初始化orb环境
orb = CORBA.ORB_init()
# 获得fluent服务器会话实例
fluentUnit = orb.string_to_object(aasFilePath.open("r").read())
# 获得scheme脚本控制器实例
scheme = fluentUnit.getSchemeControllerInstance()




In [ ]:
#这是进行DU97计算的设置 瞬态 转捩模型 0.116

result = scheme.execSchemeToString(r'(read-case "C:/Users/27749/Desktop/python/20220112-DU97.cas")')
print(scheme.doMenuCommandToString("/mesh/check"))
#设置计算的材料
#     scheme.doMenuCommand('define/materials/cc/air air yes ideal-gas  no no no no no no')
#先打开需要写入数据的csv文件
f = open('./DU97-W-300.csv'.format(i),'w',newline='',encoding='utf-8')
csv_writer = csv.writer(f)

csv_writer.writerow(['Attack_angle','Lift_coefficient'])

for Attack_angle in range(0,20,3):


    print('正在计算第{}个模型{}°攻角的数据'.format(i,Attack_angle))
    #将计算的角度转化为弧度
    Attack_angle_rad = Attack_angle*np.pi/180
    
    #设置计算湍流模型
    
    #设置边界条件
    scheme.doMenuCommand('define/bc/set/vi  inlet () vmag  no 45 d0 no {} d1  no {}  quit'.format(np.cos(Attack_angle_rad),np.sin(Attack_angle_rad)))
    #设置为瞬态计算 首先设置为一阶瞬态
    scheme.doMenuCommand("define/models/u1o yes")      




    #设置离散格式
    scheme.doMenuCommand('solve/set/disc/pre 11 mom 0 k 4 in 4 re 4 ome 4 ')
    #设置warped
    scheme.doMenuCommand('solve/set/wa enable yes yes ')#fast mode
    #设置库朗数
    scheme.doMenuCommand('solve/set/pvcontrols 20 0.75 0.75 ')


    #初始化流场
    if Attack_angle == 0:
        scheme.doMenuCommand('/solve/initialize/compute-defaults/ vi inlet')
        scheme.doMenuCommand('/solve/initialize/initialize-flow ok')


    #给出计算步数以及步幅
    scheme.doMenuCommand('solve/set/transient-controls/time-step-size 0.1')
    scheme.doMenuCommand('/solve/dual-time-iterate 10  20')#这步执行后直接开始计算




    #设置初始参考值 这个参考值只是影响最终的后处理 并不会影响计算过程
    scheme.doMenuCommand('report/reference-values area 1 length 1 depth 1 pressure 101325')
    scheme.doMenuCommand('report/reference-values/compute pff far-field')    

    #计算的输出的力的角度是可以改变的
    forces = scheme.doMenuCommandToString('/report/forces/wall-forces yes  {} {} no'.format(-np.sin(Attack_angle_rad),np.cos(Attack_angle_rad)))
    #先将文件存储为txt文件 方便最终的读取特定行
    with open('./Naca0012_data.txt','w') as fp:
        fp.write(forces)
    #第十二行是存储力的数据的行 并且将原来的字符串来利用空格分隔  
    data = linecache.getline('./Naca0012_data.txt',12).split()
    #需要将前面进行处理的文件进行清除 否则一直存在一个缓存数据 导致每次获取的数据都是一样的
    linecache.clearcache()

    row_data = [Attack_angle,data[4]]
    #将最终的数据写入csv文件
    csv_writer.writerow(row_data)
f.close()

print('计算完毕')